# ⚡ Day 1 — Exploratory Data Analysis (EDA)
## Energy Consumption Forecasting | Claysys AI Hackathon 2026

**Date:** February 19, 2026  
**Objective:** Understand the dataset structure, identify patterns, missing values, seasonality, and trends in household energy consumption.

---

## 1. Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'figure.dpi': 120, 'font.size': 10})

print('✅ Libraries imported successfully')
print(f'   pandas  : {pd.__version__}')
print(f'   numpy   : {np.__version__}')

## 2. Load Dataset

In [ ]:
# Update this path to point to your dataset
DATA_PATH = '../data/raw/household_power_consumption.txt'

df = pd.read_csv(
    DATA_PATH,
    sep=';',
    na_values='?',
    low_memory=False,
    dtype={
        'Global_active_power': 'float32',
        'Global_reactive_power': 'float32',
        'Voltage': 'float32',
        'Global_intensity': 'float32',
        'Sub_metering_1': 'float32',
        'Sub_metering_2': 'float32',
        'Sub_metering_3': 'float32',
    }
)

# Parse datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df.drop(columns=['Date', 'Time'], inplace=True)
df.set_index('Datetime', inplace=True)
df.sort_index(inplace=True)

print(f'✅ Dataset loaded!')
print(f'   Shape         : {df.shape}')
print(f'   Date range    : {df.index.min()}  →  {df.index.max()}')
print(f'   Duration      : {(df.index.max() - df.index.min()).days} days')
print(f'   Memory usage  : {df.memory_usage(deep=True).sum() / 1e6:.1f} MB')

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Statistical summary
df.describe().T.style.background_gradient(cmap='Blues')

## 3. Missing Values Analysis

In [ ]:
missing = pd.DataFrame({
    'Missing Count': df.isna().sum(),
    'Missing %': (df.isna().sum() / len(df) * 100).round(2),
})
print('Missing Values Summary:')
print(missing)
print(f'\nTotal missing: {df.isna().sum().sum():,} / {df.size:,} ({df.isna().sum().sum()/df.size*100:.2f}%)')

In [ ]:
# Visualize missing data
fig, ax = plt.subplots(figsize=(10, 3))
missing_pct = df.isna().mean() * 100
bars = ax.bar(missing_pct.index, missing_pct.values, color='tomato', alpha=0.8)
ax.bar_label(bars, fmt='%.2f%%', padding=3, fontsize=9)
ax.set_title('Missing Data Percentage per Feature', fontweight='bold')
ax.set_ylabel('Missing %')
ax.set_ylim(0, missing_pct.max() * 1.4)
plt.xticks(rotation=20, ha='right')
plt.tight_layout()
plt.savefig('../reports/figures/missing_values.png', bbox_inches='tight')
plt.show()

## 4. Full Time Series Overview

In [ ]:
# Resample to daily averages for overview plot
daily = df.resample('D').mean(numeric_only=True)

fig, axes = plt.subplots(4, 2, figsize=(16, 14))
axes = axes.flatten()

colors = ['#2196F3', '#FF9800', '#4CAF50', '#9C27B0', '#F44336', '#00BCD4', '#795548']
cols   = df.columns.tolist()

for i, col in enumerate(cols):
    axes[i].plot(daily.index, daily[col], color=colors[i], linewidth=0.8, alpha=0.85)
    axes[i].set_title(col, fontweight='bold', fontsize=10)
    axes[i].set_ylabel('Daily Average')
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Hide last empty subplot
axes[-1].set_visible(False)

fig.suptitle('Daily Average — All Features (Dec 2006 – Nov 2010)', fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../reports/figures/all_features_daily.png', bbox_inches='tight')
plt.show()

## 5. Target Variable Deep Dive — Global Active Power

In [ ]:
target = df['Global_active_power'].dropna()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# --- Full time series (hourly resampled) ---
hourly = target.resample('h').mean()
axes[0, 0].plot(hourly.index, hourly.values, linewidth=0.5, color='steelblue', alpha=0.8)
axes[0, 0].set_title('Global Active Power — Hourly Avg (Full Dataset)', fontweight='bold')
axes[0, 0].set_ylabel('kW')

# --- Distribution ---
axes[0, 1].hist(target.dropna(), bins=100, color='steelblue', edgecolor='white', alpha=0.85)
axes[0, 1].axvline(target.mean(), color='red', linestyle='--', linewidth=1.5, label=f'Mean: {target.mean():.2f}')
axes[0, 1].axvline(target.median(), color='orange', linestyle='--', linewidth=1.5, label=f'Median: {target.median():.2f}')
axes[0, 1].set_title('Distribution of Global Active Power', fontweight='bold')
axes[0, 1].set_xlabel('kW')
axes[0, 1].legend()

# --- Monthly averages ---
monthly = target.resample('ME').mean()
axes[1, 0].bar(monthly.index, monthly.values, color='steelblue', alpha=0.8, width=20)
axes[1, 0].set_title('Monthly Average Global Active Power', fontweight='bold')
axes[1, 0].set_ylabel('kW')
axes[1, 0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(axes[1, 0].get_xticklabels(), rotation=45)

# --- Hourly pattern (average by hour of day) ---
df_temp = df.copy()
df_temp['hour'] = df_temp.index.hour
hourly_avg = df_temp.groupby('hour')['Global_active_power'].mean()
axes[1, 1].plot(hourly_avg.index, hourly_avg.values, 'o-', color='steelblue', linewidth=2)
axes[1, 1].fill_between(hourly_avg.index, hourly_avg.values, alpha=0.15, color='steelblue')
axes[1, 1].set_title('Average Power Consumption by Hour of Day', fontweight='bold')
axes[1, 1].set_xlabel('Hour')
axes[1, 1].set_ylabel('kW')
axes[1, 1].set_xticks(range(0, 24))

fig.suptitle('Global Active Power — Deep Dive Analysis', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/global_active_power_analysis.png', bbox_inches='tight')
plt.show()

## 6. Seasonal Patterns

In [ ]:
df_temp = df.copy()
df_temp['hour']      = df_temp.index.hour
df_temp['dayofweek'] = df_temp.index.dayofweek
df_temp['month']     = df_temp.index.month

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# By hour
hour_avg = df_temp.groupby('hour')['Global_active_power'].mean()
axes[0].plot(hour_avg.index, hour_avg.values, 'o-', lw=2, color='#2196F3')
axes[0].fill_between(hour_avg.index, hour_avg.values, alpha=0.15, color='#2196F3')
axes[0].set_title('By Hour of Day', fontweight='bold')
axes[0].set_xlabel('Hour')
axes[0].set_ylabel('Avg Global Active Power (kW)')

# By day of week
dow_avg = df_temp.groupby('dayofweek')['Global_active_power'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
colors_dow = ['#2196F3'] * 5 + ['#FF9800'] * 2
axes[1].bar(days, dow_avg.values, color=colors_dow, alpha=0.85)
axes[1].set_title('By Day of Week', fontweight='bold')
axes[1].set_ylabel('Avg Global Active Power (kW)')

# By month
month_avg = df_temp.groupby('month')['Global_active_power'].mean()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[2].bar(months, month_avg.values, color='#4CAF50', alpha=0.85)
axes[2].set_title('By Month', fontweight='bold')
axes[2].set_ylabel('Avg Global Active Power (kW)')

fig.suptitle('Energy Consumption Patterns by Time Period', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/seasonal_patterns.png', bbox_inches='tight')
plt.show()

## 7. Correlation Analysis

In [ ]:
# Correlation heatmap
corr = df.corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(9, 7))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(
    corr, annot=True, fmt='.2f', mask=mask,
    cmap='coolwarm', center=0,
    linewidths=0.5, ax=ax,
    annot_kws={'size': 9}
)
ax.set_title('Feature Correlation Matrix', fontweight='bold', fontsize=12)
plt.tight_layout()
plt.savefig('../reports/figures/correlation_heatmap.png', bbox_inches='tight')
plt.show()

# Top correlations with target
print('\nCorrelations with Global_active_power:')
print(corr['Global_active_power'].sort_values(ascending=False))

## 8. Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Use hourly resampled data for decomposition
hourly_series = df['Global_active_power'].resample('h').mean().fillna(method='ffill')

# Decompose with 24-hour period (daily seasonality pattern)
decomp = seasonal_decompose(hourly_series.iloc[:8760], model='additive', period=24)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))
decomp.observed.plot(ax=axes[0], color='steelblue'); axes[0].set_title('Observed', fontweight='bold')
decomp.trend.plot(ax=axes[1], color='darkorange'); axes[1].set_title('Trend', fontweight='bold')
decomp.seasonal.plot(ax=axes[2], color='green'); axes[2].set_title('Seasonal (Daily)', fontweight='bold')
decomp.resid.plot(ax=axes[3], color='red', alpha=0.6); axes[3].set_title('Residual', fontweight='bold')

for ax in axes:
    ax.set_ylabel('kW')

fig.suptitle('Seasonal Decomposition of Global Active Power (First Year)', fontweight='bold', fontsize=13)
plt.tight_layout()
plt.savefig('../reports/figures/seasonal_decomposition.png', bbox_inches='tight')
plt.show()

## 9. Sub-Metering Analysis

In [ ]:
sub_daily = df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].resample('D').mean()

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(sub_daily['Sub_metering_1'], label='Kitchen', linewidth=0.7, alpha=0.8, color='#F44336')
ax.plot(sub_daily['Sub_metering_2'], label='Laundry Room', linewidth=0.7, alpha=0.8, color='#2196F3')
ax.plot(sub_daily['Sub_metering_3'], label='Water Heater & A/C', linewidth=0.7, alpha=0.8, color='#4CAF50')
ax.set_title('Daily Average Sub-Metering Consumption by Area', fontweight='bold', fontsize=12)
ax.set_ylabel('Watt-hour')
ax.legend(loc='upper right')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig('../reports/figures/sub_metering_analysis.png', bbox_inches='tight')
plt.show()

## 10. Key EDA Findings & Observations

### 📊 Dataset Summary
| Metric | Value |
|--------|-------|
| Total records | 2,075,259 |
| Time range | Dec 2006 – Nov 2010 (~4 years) |
| Granularity | 1-minute intervals |
| Missing values | ~1.25% |
| Target mean | ~1.09 kW |

### 🔍 Key Observations
1. **Strong daily seasonality**: Peak usage at morning (7–9am) and evening (6–10pm)
2. **Weekly pattern**: Weekend consumption differs from weekdays
3. **Annual seasonality**: Higher consumption in winter months (heating)
4. **High correlation**: Global_active_power strongly correlates with Global_intensity (>0.99)
5. **Sub-metering 3** (water heater) is the largest contributor

### 📋 Next Steps (Day 2)
- Handle missing values via linear interpolation
- Resample to hourly frequency (more tractable)
- Engineer lag and rolling features
- Encode cyclical time features

In [ ]:
# Save processed summary stats for Day 2
summary_stats = df.describe().T
summary_stats.to_csv('../data/processed/eda_summary_stats.csv')
print('✅ Summary stats saved to data/processed/eda_summary_stats.csv')
print('\n🎉 Day 1 EDA Complete! Ready for Day 2: Preprocessing & Feature Engineering')